<a href="https://colab.research.google.com/github/ntatfff/bert-demo/blob/main/BERT_HAM_SPAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tensorflow tensorflow_text tensorflow_hub pandas

In [4]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import pandas as pd

In [5]:
data= 'https://raw.githubusercontent.com/ntatfff/bert-demo/main/spam_ham_dataset.csv'
df_all = pd.read_csv(data)

In [6]:
# get trainning dataset with 50 spam and 50 ham
df_50_spam = df_all[df_all['label'] == 'spam'][:50]
df_50_ham = df_all[df_all['label'] == 'ham'][:50]
frames = [df_50_spam, df_50_ham]
df_train = pd.concat(frames)

In [11]:
dataset = df_train['text']
labels = df_train['label_num']

0    0
1    0
2    0
3    1
4    0
Name: label_num, dtype: int64

In [12]:
# Use train_test_split to split our data into train and validation sets
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(dataset, labels, random_state=42, test_size=0.1)

In [13]:
# Load the BERT model and tokenizer
bert_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_encoder = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
preprocessor = hub.KerasLayer(bert_preprocess)
encoder = hub.KerasLayer(bert_encoder)

In [15]:
# Prepare the model
def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(bert_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(bert_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

classifier_model = build_classifier_model()

In [16]:
# Compile the model
classifier_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# Train the model
history = classifier_model.fit(
    train_inputs,
    train_labels,
    epochs=10,
    validation_data=(validation_inputs, validation_labels)
)

Epoch 1/10
3/3 [==============================] - 153s 44s/step - loss: 1.1207 - accuracy: 0.4333 - val_loss: 0.6759 - val_accuracy: 0.6000
Epoch 2/10
3/3 [==============================] - 122s 40s/step - loss: 0.7685 - accuracy: 0.5222 - val_loss: 0.7627 - val_accuracy: 0.4000
Epoch 3/10


In [ ]:
df_test = df_all.iloc[101:120]
texts_test = df_test['text']
labels_test = df_test['label_num']


In [ ]:
# Evaluate the model
loss, accuracy = classifier_model.evaluate(texts_test, labels_test)

1/1 [==============================] - 6s 6s/step - loss: 0.6402 - accuracy: 0.6842


In [ ]:
df_predict = df_all.iloc[121:220]
reviews = df_predict['text']
classifier_model.predict(reviews)

4/4 [==============================] - 39s 9s/step


array([[-0.3901175 ],
       [-0.39005587],
       [-0.39017764],
       [-0.39016923],
       [-0.390067  ],
       [-0.39012608],
       [-0.39012066],
       [-0.39006016],
       [-0.39022353],
       [-0.3900806 ],
       [-0.39004406],
       [-0.39023367],
       [-0.39004645],
       [-0.39003488],
       [-0.3901241 ],
       [-0.39012143],
       [-0.39012417],
       [-0.39013848],
       [-0.38998446],
       [-0.3900697 ],
       [-0.3901144 ],
       [-0.3900561 ],
       [-0.39004937],
       [-0.39030716],
       [-0.39008692],
       [-0.39008877],
       [-0.39017567],
       [-0.3901249 ],
       [-0.39005753],
       [-0.39004204],
       [-0.39011696],
       [-0.3900809 ],
       [-0.3902597 ],
       [-0.3903815 ],
       [-0.3900797 ],
       [-0.3900257 ],
       [-0.39008626],
       [-0.39012644],
       [-0.39007583],
       [-0.39003477],
       [-0.39017966],
       [-0.3903651 ],
       [-0.389995  ],
       [-0.39007244],
       [-0.39009967],
       [-0

In [ ]:
df.groupby(['label'])['label'].count()
df_predict.groupby(['label'])['label'].count()

label
ham     68
spam    31
Name: label, dtype: int64